In [54]:
import pandas as pd
# https://github.com/derek73/python-nameparser
from nameparser import HumanName

In [55]:
df = pd.read_csv("../data/transinq_results.csv")

/Users/sahluwalia/acre/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3257: DtypeWarning: Columns (21,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
df['Date Occured'] = pd.to_datetime(df['Date Occured'])

In [57]:
df['year_of_donation'] = df['Date Occured'].dt.year

In [58]:
df = df.rename(columns={"Employer's Name/Specific Field": "employer"})

In [59]:
df.columns

Index(['Name', 'Street Line 1', 'Street Line 2', 'City', 'State', 'Zip Code',
       'Profession/Job Title', 'employer', 'Transction Type', 'Committee Name',
       'Committee SBoE ID', 'Committee Street 1', 'Committee Street 2',
       'Committee City', 'Committee State', 'Committee Zip Code',
       'Report Name', 'Date Occured', 'Account Code', 'Amount',
       'Form of Payment', 'Purpose', 'Candidate/Referendum Name',
       'Declaration', 'year_of_donation'],
      dtype='object')

In [60]:
flattened = df.groupby(['Name','year_of_donation', 'City', 'State', 'Zip Code', 'employer',
        'Profession/Job Title', 'Committee Name']).sum()['Amount'].reset_index()

In [61]:
flattened['Zip Code'] = flattened['Zip Code'].apply(lambda x: str(x)[:5])

In [62]:
flattened['first_name'] = flattened['Name'].apply(lambda x: HumanName(str(x))['first'])
flattened['last_name'] = flattened['Name'].apply(lambda x: HumanName(str(x))['last'])

In [63]:
flattened.shape

(297164, 11)

In [64]:
flattened.first_name.nunique()

12957

In [65]:
flattened[(flattened['year_of_donation'] == 2020) & (flattened['Amount'] > 0)].shape

(44855, 11)

In [67]:
flattened.groupby(['City', 'State', 'Zip Code', 'employer',
        'Profession/Job Title', 'Committee Name', 'first_name', 'last_name']).nunique()['year_of_donation'].shape

(167272,)

In [72]:
flattened['last_name'] = flattened['last_name'].str.lower()

flattened['first_name'] = flattened['first_name'].str.lower()

In [ ]:
# confirms that some NC voters did not donate in every year

In [74]:
flattened.to_csv('../data/nc_contributions_clean.csv')